In [151]:
# torch 의 유용한 함수들
import numpy as np
import torch
import torch.nn as nn  # neural network 모음. (e.g. nn.Linear, nn.Conv2d, BatchNorm, Loss functions 등등)
import torch.optim as optim  # Optimization algorithm 모음, (e.g. SGD, Adam, 등등)
import torch.nn.functional as F  # 파라미터가 필요없는 Function 모음
import torchvision.datasets as datasets  # 표준 데이터 세트 모음
import torchvision.transforms as transforms  # 데이터 세트에 적용 할 수있는 변환 관련 함수 모음
from torch.utils.data import DataLoader  # 데이터 세트 관리 및 미니 배치 생성을 위한 함수 모음
from torch.utils.data import TensorDataset # 텐서데이터셋
import torch.backends.cudnn as cudnn  # cudnn을 다루기 위한 값 모음
import torch.onnx  # model을 onnx 로 변환하기 위함

In [92]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Linear Regression Model

In [93]:
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[5], [7], [9]])

In [94]:
x_train, x_train.shape

(tensor([[1.],
         [2.],
         [3.]]),
 torch.Size([3, 1]))

In [95]:
# 가중치 설정
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [96]:
W, W.shape

(tensor([0.], requires_grad=True), torch.Size([1]))

In [97]:
hypothesis = x_train * W + b

In [98]:
hypothesis

tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)

In [99]:
(hypothesis - y_train) ** 2 / 3

tensor([[ 8.3333],
        [16.3333],
        [27.0000]], grad_fn=<DivBackward0>)

In [100]:
cost = torch.mean((hypothesis - y_train) ** 2)
print(cost)

tensor(51.6667, grad_fn=<MeanBackward0>)


In [101]:
W

tensor([0.], requires_grad=True)

In [102]:
optimizer = optim.SGD([W, b], lr=0.01)

In [103]:
optimizer.zero_grad()
cost.backward()
optimizer.step()

### 1 epoch 실행 시 W 변화

In [104]:
W

tensor([0.3067], requires_grad=True)

In [105]:
epochs = 3000

for epoch in range(epochs+1):
    
    hypothesis = x_train * W + b
    cost = torch.mean((hypothesis - y_train) ** 2)

    optimizer.zero_grad() # RNN에서는 전파과정에 gradient add 동작 존재
    cost.backward()
    optimizer.step()

    if epoch % 200 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, epochs, W.item(), b.item(), cost.item()
            ))

Epoch    0/3000 W: 0.579, b: 0.265 Cost: 40.932430
Epoch  200/3000 W: 2.481, b: 1.908 Cost: 0.172035
Epoch  400/3000 W: 2.297, b: 2.325 Cost: 0.065691
Epoch  600/3000 W: 2.184, b: 2.583 Cost: 0.025084
Epoch  800/3000 W: 2.113, b: 2.742 Cost: 0.009578
Epoch 1000/3000 W: 2.070, b: 2.841 Cost: 0.003658
Epoch 1200/3000 W: 2.043, b: 2.902 Cost: 0.001397
Epoch 1400/3000 W: 2.027, b: 2.939 Cost: 0.000533
Epoch 1600/3000 W: 2.017, b: 2.962 Cost: 0.000204
Epoch 1800/3000 W: 2.010, b: 2.977 Cost: 0.000078
Epoch 2000/3000 W: 2.006, b: 2.986 Cost: 0.000030
Epoch 2200/3000 W: 2.004, b: 2.991 Cost: 0.000011
Epoch 2400/3000 W: 2.002, b: 2.995 Cost: 0.000004
Epoch 2600/3000 W: 2.001, b: 2.997 Cost: 0.000002
Epoch 2800/3000 W: 2.001, b: 2.998 Cost: 0.000001
Epoch 3000/3000 W: 2.001, b: 2.999 Cost: 0.000000


### pytorch Model 활용 Linear Regression Model

In [112]:
model = nn.Linear(1, 1)
list(model.parameters())

[Parameter containing:
 tensor([[-0.1806]], requires_grad=True),
 Parameter containing:
 tensor([0.9801], requires_grad=True)]

In [113]:
x_train, model(x_train)

(tensor([[1.],
         [2.],
         [3.]]),
 tensor([[0.7995],
         [0.6189],
         [0.4382]], grad_fn=<AddmmBackward0>))

In [114]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [115]:
epochs = 3000

for epoch in range(epochs+1):
    
    prediction = model(x_train)
    cost = F.mse_loss(prediction, y_train) # torch.mean((hypothesis - y_train) ** 2)

    optimizer.zero_grad() 
    cost.backward()
    optimizer.step()

    if epoch % 200 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, epochs, cost.item()
            ))

Epoch    0/3000 Cost: 43.888977
Epoch  200/3000 Cost: 0.043259
Epoch  400/3000 Cost: 0.016519
Epoch  600/3000 Cost: 0.006308
Epoch  800/3000 Cost: 0.002409
Epoch 1000/3000 Cost: 0.000920
Epoch 1200/3000 Cost: 0.000351
Epoch 1400/3000 Cost: 0.000134
Epoch 1600/3000 Cost: 0.000051
Epoch 1800/3000 Cost: 0.000020
Epoch 2000/3000 Cost: 0.000007
Epoch 2200/3000 Cost: 0.000003
Epoch 2400/3000 Cost: 0.000001
Epoch 2600/3000 Cost: 0.000000
Epoch 2800/3000 Cost: 0.000000
Epoch 3000/3000 Cost: 0.000000


In [119]:
model(x_train), y_train, list(model.parameters())

(tensor([[4.9996],
         [6.9999],
         [9.0002]], grad_fn=<AddmmBackward0>),
 tensor([[5.],
         [7.],
         [9.]]),
 [Parameter containing:
  tensor([[2.0003]], requires_grad=True),
  Parameter containing:
  tensor([2.9993], requires_grad=True)])

### pytorch Model 활용 Multi Linear Regression Model

In [133]:
x_train = torch.FloatTensor([[73, 80, 75], [93, 88, 93], [89, 91, 90], [96, 98, 100], [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [134]:
x_train.shape, y_train.shape

(torch.Size([5, 3]), torch.Size([5, 1]))

In [135]:
model = nn.Linear(3, 1)

In [136]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.0071,  0.5706, -0.2668]], requires_grad=True),
 Parameter containing:
 tensor([-0.5093], requires_grad=True)]

In [137]:
optimizer = optim.SGD(model.parameters(), lr=1e-5)

In [138]:
epochs = 6000

for epoch in range(epochs+1):
    
    prediction = model(x_train)
    cost = F.mse_loss(prediction, y_train) # torch.mean((hypothesis - y_train) ** 2)

    optimizer.zero_grad() 
    cost.backward()
    optimizer.step()

    if epoch % 600 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, epochs, cost.item()
            ))

Epoch    0/6000 Cost: 21828.746094
Epoch  600/6000 Cost: 6.441886
Epoch 1200/6000 Cost: 4.704942
Epoch 1800/6000 Cost: 3.450146
Epoch 2400/6000 Cost: 2.543673
Epoch 3000/6000 Cost: 1.888818
Epoch 3600/6000 Cost: 1.415742
Epoch 4200/6000 Cost: 1.073956
Epoch 4800/6000 Cost: 0.827039
Epoch 5400/6000 Cost: 0.648667
Epoch 6000/6000 Cost: 0.519758


In [140]:
model(x_train), y_train, list(model.parameters())

(tensor([[152.2929],
         [184.0934],
         [180.9597],
         [196.3176],
         [141.1826]], grad_fn=<AddmmBackward0>),
 tensor([[152.],
         [185.],
         [180.],
         [196.],
         [142.]]),
 [Parameter containing:
  tensor([[0.9647, 0.6484, 0.4067]], requires_grad=True),
  Parameter containing:
  tensor([-0.4972], requires_grad=True)])

#### Class을 이용한 Model 구현

In [144]:
class MultiLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
    
    def forward(self, inputdata):
        return self.linear(inputdata)

In [145]:
model = MultiLinearRegressionModel()

In [146]:
model # model의 Layer 정보

MultiLinearRegressionModel(
  (linear): Linear(in_features=3, out_features=1, bias=True)
)

In [147]:
optimizer = optim.SGD(model.parameters(), lr=1e-5)

In [148]:
epochs = 3000

for epoch in range(epochs+1):
    
    prediction = model.forward(x_train)
    cost = F.mse_loss(prediction, y_train)

    optimizer.zero_grad() 
    cost.backward()
    optimizer.step()

    if epoch % 600 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, epochs, cost.item()
            ))

Epoch    0/3000 Cost: 53096.925781
Epoch  600/3000 Cost: 0.581358
Epoch 1200/3000 Cost: 0.506776
Epoch 1800/3000 Cost: 0.451699
Epoch 2400/3000 Cost: 0.410761
Epoch 3000/3000 Cost: 0.380049


In [149]:
model(x_train), y_train, list(model.parameters())

(tensor([[151.5666],
         [184.5262],
         [180.5832],
         [196.7709],
         [141.2561]], grad_fn=<AddmmBackward0>),
 tensor([[152.],
         [185.],
         [180.],
         [196.],
         [142.]]),
 [Parameter containing:
  tensor([[0.8170, 0.4834, 0.7085]], requires_grad=True),
  Parameter containing:
  tensor([0.1164], requires_grad=True)])

#### Pytorch Dataset 이용

In [152]:
x_train  =  torch.FloatTensor([[73,  80,  75],
                               [93,  88,  93],
                               [89,  91,  90],
                               [96,  98,  100],
                               [73,  66,  70]])
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

dataset = TensorDataset(x_train, y_train)

In [157]:
dataset, len(dataset), dataset[0], dataset[2], dataset[3]

(<torch.utils.data.dataset.TensorDataset at 0x176703690>,
 5,
 (tensor([73., 80., 75.]), tensor([152.])),
 (tensor([89., 91., 90.]), tensor([180.])),
 (tensor([ 96.,  98., 100.]), tensor([196.])))

In [185]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [186]:
dataloader, len(dataloader)

for index, batchdata in enumerate(dataloader):
    print(batchdata[0], batchdata[1])

tensor([[89., 91., 90.],
        [73., 80., 75.]]) tensor([[180.],
        [152.]])
tensor([[93., 88., 93.],
        [73., 66., 70.]]) tensor([[185.],
        [142.]])
tensor([[ 96.,  98., 100.]]) tensor([[196.]])


In [206]:
model = nn.Linear(3, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)

In [207]:
epochs = 60
for epoch in range(epochs+1):
    for index, batchdata in enumerate(dataloader):
        
        prediction = model.forward(batchdata[0])
        cost = F.mse_loss(prediction, batchdata[1])

        optimizer.zero_grad() 
        cost.backward()
        optimizer.step()
        
    if epoch % 10 == 0:
        print(
            "Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}".format(
                epoch, epochs, index + 1, len(dataloader), cost.item()
            )
        )

Epoch    0/60 Batch 3/3 Cost: 62435.906250
Epoch   10/60 Batch 3/3 Cost: 4106.858887
Epoch   20/60 Batch 3/3 Cost: 129.297607
Epoch   30/60 Batch 3/3 Cost: 7.957520
Epoch   40/60 Batch 3/3 Cost: 0.016834
Epoch   50/60 Batch 3/3 Cost: 10.291803
Epoch   60/60 Batch 3/3 Cost: 8.525053


#### Custom Dataset 사용 

In [244]:
x_train  =  np.array([[73,  80,  75],
                               [93,  88,  93],
                               [89,  91,  90],
                               [96,  98,  100],
                               [73,  66,  70]], dtype=float)
y_train  =  np.array([[152],  [185],  [180],  [196],  [142]], dtype=float)

In [245]:
x_train.shape, y_train.shape

((5, 3), (5, 1))

In [246]:
class CustomDataset(TensorDataset):
    def __init__(self, x_train, y_train):
        super().__init__()
        self.x_train = x_train
        self.y_train = y_train

    def __len__(self):
        return len(x_train)
    
    def __getitem__(self, index):
        return torch.FloatTensor(self.x_train[index]), torch.FloatTensor(self.y_train[index])

In [247]:
customdataset = CustomDataset(x_train, y_train)

In [236]:
customdataset, len(customdataset), customdataset[0]

(<__main__.CustomDataset at 0x1778c8a10>,
 5,
 (tensor([73., 80., 75.]), tensor([152.])))

In [248]:
dataloader = DataLoader(customdataset, batch_size=2, shuffle=True)

In [249]:
model = torch.nn.Linear(3, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

In [250]:
epochs = 60
for epoch in range(epochs+1):
    for index, batchdata in enumerate(dataloader):

        prediction = model.forward(batchdata[0])
        cost = F.mse_loss(prediction, batchdata[1])

        optimizer.zero_grad() 
        cost.backward()
        optimizer.step()
        
    if epoch % 10 == 0:
        print(
            "Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}".format(
                epoch, epochs, index + 1, len(dataloader), cost.item()
            )
        )

Epoch    0/60 Batch 3/3 Cost: 11844.422852
Epoch   10/60 Batch 3/3 Cost: 0.009883
Epoch   20/60 Batch 3/3 Cost: 0.179863
Epoch   30/60 Batch 3/3 Cost: 0.850923
Epoch   40/60 Batch 3/3 Cost: 0.000000
Epoch   50/60 Batch 3/3 Cost: 0.145298
Epoch   60/60 Batch 3/3 Cost: 0.661496
